# GEFS Realtime Combiner Script
## Robert Fritzen - NIU WCS

This script (notebook) is used to generate WRF intermediate files from realtime GEFS data that can then be used to initialize downscaled WRF runs

## Imports
* xarray
* numpy
* pygrib
* pywinter (pip3 install pywinter -> Needs gfortran)

In [20]:
import xarray as xr
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import pygrib
import pickle

# pip3 install pywinter (Need gfortran installed)
from pywinter import winter

In [25]:
# Format: YYYYMMDDHH
init_date = "2021071200"   #GEFS data are available for four days from the current date
start_date = "2021071200"
end_date = "2021071600"

# Time delta of GEFS (3 hours)
dt = 3

# Which member of the GEFS reforecast set to use (Options are c00, p01 - p30)
dataMember = "c00"

# Data source (Leave this as is unless the source is moved (Will require an edit of the next block))
dataSrc = "https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/"

In [26]:
sLevels = ['000010', '010040', '040100', '100200']

# These are the list of pressure levels we need to fill
pres = [100000., 97500., 95000., 92500., 90000., 85000., 80000., 75000., 70000., 65000., 60000., 55000., 50000., 
        45000., 40000., 30000., 35000., 25000., 20000., 15000., 10000., 7000., 5000., 3000., 2000., 1000., 700., 
        500., 300., 200., 100.]

variables = {
    "PMSL":
        {'storedName': 'Pressure reduced to MSL',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'meanSea',
         'iFileName' : 'PMSL',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},     
    
    "TT":
        {'storedName': 'Temperature',
         'storedFile': ['a', 'b'],
         'gribTypeOfLevel': 'isobaricInhPa',
         'iFileName' : 'TT',
         'isSoilHgt': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},

    "UU":
        {'storedName': 'U component of wind',
         'storedFile': ['a', 'b'],
         'gribTypeOfLevel': 'isobaricInhPa',
         'iFileName' : 'UU',
         'isSoilHgt': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},  

    "VV":
        {'storedName': 'V component of wind',
         'storedFile': ['a', 'b'],
         'gribTypeOfLevel': 'isobaricInhPa',
         'iFileName' : 'VV',
         'isSoilHgt': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},

    "SPECHUMD":
        {'storedName': 'Specific humidity',
         'storedFile': ['b'],
         'gribTypeOfLevel': 'isobaricInhPa',
         'iFileName' : 'SPECHUMD',
         'isSoilHgt': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},   

    "GHT":
        {'storedName': 'Geopotential Height',
         'storedFile': ['a', 'b'],
         'gribTypeOfLevel': 'isobaricInhPa',
         'iFileName' : 'GHT',
         'isSoilHgt': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},     
    
    "T2":
        {'storedName': '2 metre temperature',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'heightAboveGround',
         'iFileName' : 'TT',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},    
    
    "Q2":
        {'storedName': '2 metre specific humidity',
         'storedFile': ['b'],
         'gribTypeOfLevel': 'heightAboveGround',
         'iFileName' : 'SPECHUMD',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},   
    
    "U10":
        {'storedName': '10 metre U wind component',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'heightAboveGround',
         'iFileName' : 'UU',
         'isSoilHgt': False,
         'desiredLevel': None,
         'is3D': False,
         'isLandSeaMask': False,
         'isSoilVar': False,
         'NANOverWater': False,
         'NeedFlipUD': True},     

    "V10":
        {'storedName': '10 metre V wind component',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'heightAboveGround',
         'iFileName' : 'VV',
         'isSoilHgt': False,
         'desiredLevel': None,
         'is3D': False,
         'isLandSeaMask': False,
         'isSoilVar': False,
         'NANOverWater': False,
         'NeedFlipUD': True},    

    "PSFC":
        {'storedName': 'Surface pressure',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'surface',
         'iFileName' : 'PSFC',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},              

    "SM":
        {'storedName': 'Volumetric soil moisture content',
         'storedFile': ['a', 'b'],
         'gribTypeOfLevel': 'depthBelowLandLayer',
         'iFileName' : 'SM',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': True,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'isSoilMoisture': True,
         'isSoilTemperature': False,
         'NANOverWater': True,
         'NeedFlipUD': True}, 
    
    "ST":
        {'storedName': 'Soil Temperature',
         'storedFile': ['a', 'b'],
         'gribTypeOfLevel': 'depthBelowLandLayer',
         'iFileName' : 'ST',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': True,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'isSoilMoisture': False,
         'isSoilTemperature': True,
         'NANOverWater': True,
         'NeedFlipUD': True},     
    
    # LANDSEA is a special case, handled within the loop
    "LANDSEA":
        {'storedName': 'Land-sea mask',
         'storedFile': ['b'],
         'gribTypeOfLevel': 'surface',
         'iFileName': 'LANDSEA',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': True,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},
    
    # To-Do Note: This comes in gpm, not m, check if ok?
    "SOILHGT":
        {'storedName': 'Orography',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'surface',
         'iFileName' : 'SOILHGT',
         'isSoilHgt': True,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},     
    
    "SKINTEMP":
        {'storedName': 'Temperature',
         'storedFile': ['b'],
         'gribTypeOfLevel': 'surface',
         'iFileName' : 'SKINTEMP',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},                    
    
    "SNOW":
        {'storedName': 'Water equivalent of accumulated snow depth (deprecated)',
         'storedFile': ['a'],
         'gribTypeOfLevel': 'surface',
         'iFileName' : 'SNOW',
         'isSoilHgt': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': True,
         'NeedFlipUD': True},                  
}

# Function definitions
def grib_to_xs(grib, vName):
    vals = grib.values
    print(" - Converting GRIB to XArray of type " + str(type(vals)))
    if(np.ma.isMaskedArray(vals)):
        print(" -- Converting Masked Numpy Array to Standard Numpy Array")
        # Convert the masked array to a standard np array
        vals = np.ma.filled(vals, np.nan)
    
    arr = xr.DataArray(vals)
    arr = arr.astype('float32')
    arr = arr.rename({'dim_0':'lat', 'dim_1':'lon'})
    xs = arr.to_dataset(name=vName)
    
    return xs

def filter_grib_layers(grbs,**grib_layer_filters):
    """
    Replacement of pygrib.select() function that filters a list of grib messages, not a grib object itself. This allows
    for multiple consecutive calls of that function since input and output of this function is both a list (except when
    there is only one grib layer left after filtering.
    :param grbs: List of grib messages
    :param grib_layer_filters: pygrib.select()-style kwargs for selecting data e.g. validityDate = 20170104, shortName="tp" ,...
    :return: List of grib messages that satisfy the criteria
    
    Source: https://gist.github.com/marjanmo/574839b78c5ee841768f0f28713961ee
    """
    if grbs == None:
        return None

    # Returns grib_objects that meet given selection criteria.
    def passes_filters(grb):
        """
        Helper function for filtering a single grib message. True if passes the filter,
        False if it doesnt satisfy anyone of the given.
        :param grb:
        :return:
        """
        for filter in grib_layer_filters:

            if isinstance(grib_layer_filters[filter], list):
                if grb[filter] not in grib_layer_filters[filter]:
                    return False
            else:
                if grb[filter] != grib_layer_filters[filter]:
                    return False

        return True

    #Filter gribs
    ok = [i for i in grbs if passes_filters(i)]

    if isinstance(ok,list) and len(ok) == 1:
        ok = ok[0]

    return ok

In [27]:
print("Running Setting Tests")
# How many files per day
hourCt = 24 / dt

# Convert the start/end times into python datetime objects
print(" - Testing incoming time settings")
init_tm = datetime.strptime(init_date, '%Y%m%d%H')
start_tm = datetime.strptime(start_date, '%Y%m%d%H')
end_tm = datetime.strptime(end_date, '%Y%m%d%H')

startD = start_tm - init_tm
endD = end_tm - init_tm
hours_start = int(startD.total_seconds() / 3600)
hours_end = int(endD.total_seconds() / 3600)

print("  => Requested window of forecast time " + str(hours_start) + " - " + str(hours_end))
if(hours_start > 384 or hours_end > 384):
    raise ValueError(" *** Error: Forecast time window exceeds 384h limit of GEFS data.")

# The list of times we need to grab
timeList = []

print(" - Setting time list arrays")
cur_tm = start_tm
while cur_tm <= end_tm:
    hDelta = int((cur_tm - init_tm).total_seconds() / 3600)
    
    timeList.append(hDelta)
    cur_tm += timedelta(hours=dt)
    
print("  => " + str(timeList))
print("Done.")

Running Setting Tests
 - Testing incoming time settings
  => Requested window of forecast time 0 - 96
 - Setting time list arrays
  => [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96]
Done.


In [28]:
print("Start")

print("Checking for required directory structure.")
if not os.path.exists(os.getcwd() + "/Input/"):
    print(" - Input directory not found, creating...")
    os.mkdir(os.getcwd() + "/Input/")
if not os.path.exists(os.getcwd() + "/Output/"):
    print(" - Output directory not found, creating...")
    os.mkdir(os.getcwd() + "/Output/")
print("Directory check passed")

lat = None
lon = None

dlat = None
dlon = None

landsea = None
soilhgt = None

prevStep = None
cleanFiles = False

print("Collecting constant fields.")

timeString0 = (init_tm + timedelta(hours=timeList[0])).strftime("%Y%m%d%H")       
dataDirectory0 = dataSrc + "gefs." + init_tm.strftime("%Y%m%d") + "/" + init_tm.strftime("%H") + "/atmos/"

lnkA = dataDirectory0 + "pgrb2ap5/ge" + dataMember + ".t00z.pgrb2a.0p50.f000"
iPathA = "ge" + dataMember + ".t00z.pgrb2a.0p50.f000"      
lnkB = dataDirectory0 + "pgrb2bp5/ge" + dataMember + ".t00z.pgrb2b.0p50.f000"
iPathB = "ge" + dataMember + ".t00z.pgrb2b.0p50.f000"

os.system("wget " + lnkA + " -O " + os.getcwd() + "/Input/" + iPathA)
os.system("wget " + lnkB + " -O " + os.getcwd() + "/Input/" + iPathB)

print("A: " + str(lnkA) + " | " + str(lnkA) + ".idx")
print("B: " + str(lnkB) + " | " + str(lnkB) + ".idx")

gribA = pygrib.open(os.getcwd() + "/Input/" + iPathA)
gribB = pygrib.open(os.getcwd() + "/Input/" + iPathB)

print("- Setting lat/lon arrays")
try:
    tmp_sel = gribA.select(name="Temperature", typeOfLevel="isobaricInhPa", level=1000)[0]
    lat, lon = tmp_sel.latlons()

    dlat = np.abs(lat[1,0]-lat[0,0])
    dlon = np.abs(lon[0,1]-lon[0,0])
    
    print("d(lat): " + str(lat[1,0]) + " - " + str(lat[0,0]) + " => " + str(dlat))
    print("d(lat): " + str(lon[0,1]) + " - " + str(lon[0,0]) + " => " + str(dlon))
    
except:
    sys.exit("- Error: Failed to find field Temperature in GRIB file to set lat/lon with, abort.")
print("- Setting land-sea mask")
try:
    tmp_sel = gribB.select(name="Land-sea mask", typeOfLevel="surface")
    landsea = tmp_sel[0]
except:
    sys.exit("- Error: Failed to locate land-sea mask in the GRIB file, abort.")
print("- Setting soil height (orography)")
try:
    tmp_sel = gribA.select(name="Orography", typeOfLevel="surface")
    soilhgt = tmp_sel[0]  
except:
    sys.exit("- Error: Failed to locate land height field in the GRIB file, abort.")
print("Constant fields set, proceed...")
    
gribA.close()
gribB.close()

for t in timeList:
    timeString = (init_tm + timedelta(hours=t)).strftime("%Y%m%d%H")
        
    dataDirectory = dataSrc + "gefs." + init_tm.strftime("%Y%m%d") + "/" + init_tm.strftime("%H") + "/atmos/"
    print("Processing " + str(t) + " hr (" + timeString + ")")
    
    print("Checking for existing file...")
    if os.path.exists(os.getcwd() + "/Output/gefs_realtime_saveif_" + init_tm.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle"):
        print("Processed data found, loading")
        with open(os.getcwd() + "/Output/gefs_realtime_saveif_" + init_tm.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle", 'rb') as handle:
            saveIF = pickle.load(handle)
    else:
        print("No file found, begin processing")
        saveIF = {}
        
        # Start by downloading the two GRIB files we need
        lnkA = dataDirectory + "pgrb2ap5/ge" + dataMember + ".t00z.pgrb2a.0p50.f" + '{:03d}'.format(t)
        iPathA = "ge" + dataMember + ".t00z.pgrb2a.0p50.f" + '{:03d}'.format(t)        
        lnkB = dataDirectory + "pgrb2bp5/ge" + dataMember + ".t00z.pgrb2b.0p50.f" + '{:03d}'.format(t)
        iPathB = "ge" + dataMember + ".t00z.pgrb2b.0p50.f" + '{:03d}'.format(t)
        
        os.system("wget " + lnkA + " -O " + os.getcwd() + "/Input/" + iPathA)
        os.system("wget " + lnkB + " -O " + os.getcwd() + "/Input/" + iPathB)
        
        print("A: " + str(lnkA) + " | " + str(lnkA) + ".idx")
        print("B: " + str(lnkB) + " | " + str(lnkB) + ".idx")
        
        gribA = pygrib.open(os.getcwd() + "/Input/" + iPathA)
        gribB = pygrib.open(os.getcwd() + "/Input/" + iPathB)         
        
        for key, item in variables.items():
            print(" - Working " + str(key))
            
            subsel_tmp_a = None
            subsel_tmp_b = None

            save = []
            if(item['is3D'] == True):
                # Open the two GRIB files
                if('a' in item['storedFile']):
                    subsel_tmp_a = gribA.select(name=item['storedName'], typeOfLevel=item['gribTypeOfLevel'])
                if('b' in item['storedFile']):
                    subsel_tmp_b = gribB.select(name=item['storedName'], typeOfLevel=item['gribTypeOfLevel'])                    
                # Loop by level
                for p in pres:
                    brk_a = filter_grib_layers(subsel_tmp_a, level=p/100)
                    brk_b = filter_grib_layers(subsel_tmp_b, level=p/100)
                    ds = None
                    if(type(brk_a) == pygrib._pygrib.gribmessage):
                        ds = grib_to_xs(brk_a, key)                       
                    elif(type(brk_b) == pygrib._pygrib.gribmessage):
                        ds = grib_to_xs(brk_b, key)
                    else:
                        print(" - Variable " + str(key) + " does not have data at " + str(p) + ", filling NaN")                    
                        ds = np.zeros((len(lat), len(lon)))  
                        
                    if(item['NANOverWater'] == True):
                        ds[key].values[np.where(landsea == 0)] = np.nan                      
                    if(item['NeedFlipUD'] == True):
                        ds[key].values = np.flipud(ds[key].values) 
                        
                    save.append(ds[key].values)
            elif(item['isLandSeaMask'] == True):
                save = landsea.values
                if(item['NeedFlipUD'] == True):
                    save = np.flipud(save)
            elif(item['isSoilHgt'] == True):
                save = soilhgt.values
                if(item['NeedFlipUD'] == True):
                    save = np.flipud(save)                
            elif(item['isSoilVar'] == True):
                # Soil is special for GEFS because it is split across both files, and pygrib does not have a "levels" filter
                #  so we need to work some string magic to extract what we need.
                if('a' in item['storedFile']):
                    subsel_tmp_a = gribA.select(name=item['storedName'], typeOfLevel=item['gribTypeOfLevel'])
                if('b' in item['storedFile']):
                    subsel_tmp_b = gribB.select(name=item['storedName'], typeOfLevel=item['gribTypeOfLevel'])  
                    
                for sl in sLevels:
                    ds = None
                    
                    lowSoil = sl[0:3]
                    hiSoil = sl[3:]
                    
                    level_text = lowSoil[0] + "." + lowSoil[1] + "-" + hiSoil[0] + "." + hiSoil[1]
                    
                    # Scan the two GRIB files to find the level we need
                    if(subsel_tmp_a != None):
                        for grbMsg in subsel_tmp_a:                        
                            sKey = str(grbMsg)    
                            start = sKey.find("levels ")+7
                            end = sKey.find("m:")-1
                            levels = sKey[start:end]
                            
                            if(levels == level_text):
                                ds = grib_to_xs(grbMsg, key)
                    if(ds == None and subsel_tmp_b != None):
                        for grbMsg in subsel_tmp_b:                        
                            sKey = str(grbMsg)    
                            start = sKey.find("levels ")+7
                            end = sKey.find("m:")-1
                            levels = sKey[start:end]
                            
                            if(levels == level_text):
                                ds = grib_to_xs(grbMsg, key)                        
                    else:
                        print("WARN: Did not locate " + str(key) + " at level " + str(level_text))
                        nans = np.zeros((len(lat), len(lon)))
                        save.append(nans)
                    
                    if(ds != None):
                        if(item['NANOverWater'] == True):
                            ds[key].values[np.where(landsea == 0)] = np.nan

                        if(item['NeedFlipUD'] == True):
                            ds[key].values = np.flipud(ds[key].values)                                

                        save.append(ds[key].values) # = np.stack(i.values, axis=0)
            else:
                # 2D variables are easy since they don't have to worry about a vertical coordinate and ALWAYS come in
                #  only one of the two files.
                if('a' in item['storedFile']):
                    subsel = gribA.select(name=item['storedName'], typeOfLevel=item['gribTypeOfLevel'])
                if('b' in item['storedFile']):
                    subsel = gribB.select(name=item['storedName'], typeOfLevel=item['gribTypeOfLevel'])               
                
                if(item['desiredLevel'] != None):
                    subsel = filter_grib_layers(subsel, level=item['desiredLevel'])
                    
                ds = grib_to_xs(subsel[0], key)
                if(item['NANOverWater'] == True):
                    ds[key].values[np.where(landsea == 0)] = np.nan 

                if(item['NeedFlipUD'] == True):
                    ds[key].values = np.flipud(ds[key].values)                                  

                save = ds[key].values

            saveOut = np.stack(save)

            saveIF[key] = {'iFileName': item['iFileName'], 
                           'isSoilVariable': item['isSoilVar'], 
                           'is3D': item['is3D'],
                           'values': saveOut}

        gribA.close()
        gribB.close()
        
        with open(os.getcwd() + "/Output/saveif_" + init_tm.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle", 'wb') as handle:
            pickle.dump(saveIF, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(" - Data saved to file")
        
    print(" - Done Collecting GRIB data")
    print("\n")
    ## Convert to numpy arrays
    print(" - Converting to INTER: File")
    print("  => Debug Info: ")
    print("  => Pres: " + str(np.array(pres)))
    print("  => sLevels: " + str(np.array(sLevels)))
    
    # Latitude is flipped in GEFS, flip it back before writting the intermediate file.
    winter_geo = winter.Geo0(lat[-1], lon[0], dlat, dlon)
    winter_vars = []
    #
    for key, item in saveIF.items():
        print("  -> Processing " + key + " | Write as: " + item['iFileName'])
        winter_save = None
        if(item['is3D'] == True):
            print("   => 3D Variable")
            winter_save = winter.V3dp(item['iFileName'], item['values'], np.array(pres))
        elif(item['isSoilVariable'] == True):
            print("   => Soil Variable")
            winter_save = winter.Vsl(item['iFileName'], item['values'], np.array(sLevels))
        else:
            print("   => 2D Variable")
            winter_save = winter.V2d(item['iFileName'], item['values'])
        
        winter_vars.append(winter_save)  

    # This is the timestep we are on
    outTime = (init_tm + timedelta(hours=t)).strftime("%Y-%m-%d_%H")

    # Write the file.
    winter.cinter('FILE', outTime, winter_geo, winter_vars, os.getcwd() + '/Output')  
    
    prevStep = t
    
    print("Done Timestep")
    
print("Cleaning")

for key, item in variables.items():
    print(" - Cleaning " + str(key))
    if(item['path'] is not None):
        for idx, iPath in enumerate(item['path']):
            iPath = iPath.replace("[timestring]", target_date_file.strftime("%Y%m%d%H"))
            path = os.getcwd() + "/Input/" + iPath
            print("Deleting " + path)
            os.system("rm " + path)
    
print("Execution Complete")

Start
Checking for required directory structure.
Directory check passed
A: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f000 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f000.idx
B: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f000 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f000.idx
- Setting lat/lon arrays
d(lat): 89.5 - 90.0 => 0.5
d(lat): 0.5 - 0.0 => 0.5
- Setting land-sea mask
- Setting soil height (orography)
Constant fields set, proceed...
Processing 0 hr (2021071200)
Checking for existing file...
No file found, begin processing
A: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f000 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

FILE:2021-07-12_03
Done Timestep
Processing 6 hr (2021071206)
Checking for existing file...
No file found, begin processing
A: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f006 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f006.idx
B: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f006 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f006.idx
 - Working PMSL
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Working TT
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndar

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

...

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

FILE:2021-07-15_21
Done Timestep
Processing 96 hr (2021071600)
Checking for existing file...
No file found, begin processing
A: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f096 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f096.idx
B: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f096 | https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.20210712/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f096.idx
 - Working PMSL
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Working TT
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.nda

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

KeyError: 'path'

In [ ]:
grb_test1 = pygrib.open("Input/gec00.t00z.pgrb2a.0p50.f048")
grb_test2 = pygrib.open("Input/gec00.t00z.pgrb2b.0p50.f048")

"""
pres_want = [100000., 97500., 95000., 92500., 90000., 85000., 80000., 75000., 70000., 65000., 60000., 55000., 50000., 
        45000., 40000., 30000., 35000., 25000., 20000., 15000., 10000., 7000., 5000., 3000., 2000., 1000., 700., 
        500., 300., 200., 100.]

out_list = []

subsel_tmp_a = grb_test1.select(name="Temperature", typeOfLevel="isobaricInhPa")
subsel_tmp_b = grb_test2.select(name="Temperature", typeOfLevel="isobaricInhPa")

subsel_a_2 = filter_grib_layers(subsel_tmp_a, level=1000)
print(subsel_a_2)
print(type(subsel_a_2) == pygrib._pygrib.gribmessage)

for p in pres_want:
    print("Scanning " + str(p))
    brk_a = filter_grib_layers(subsel_tmp_a, level=p/100)
    brk_b = filter_grib_layers(subsel_tmp_b, level=p/100)
    
    if(type(brk_a) == pygrib._pygrib.gribmessage):
        print("Found in file A")
        sv = grib_to_xs(brk_a, "TMP")
        out_list.append(sv["TMP"].values)
    elif(type(brk_b) == pygrib._pygrib.gribmessage):
        print("Found in file B")
        sv = grib_to_xs(brk_b, "TMP")
        out_list.append(sv["TMP"].values)
    else:
        print("Not found for this level.")
        
print(out_list)

"""

print(grb_test2[285])
print(grb_test2[286])
print(grb_test2[287])

subsel_st_b = grb_test2.select(name="2 metre specific humidity")
print(subsel_st_b)

for i, gribM in enumerate(subsel_st_b):
    print(str(i) + " => " + str(gribM))
    
    for g in gribM.keys():
        try:
            print(str(g) + " => " + str(gribM[g]))
        except:
            print(str(g) + " => EMPTY")
        
brk_a = filter_grib_layers(subsel_st_b, levels="0.1-0.4")
print(brk_a)